# Análisis Exploratorio de Datos (EDA)

Este notebook realiza un análisis exploratorio completo de los datos de ventas. Incluye análisis univariado (una variable), bivariado (dos variables) y multivariado (múltiples variables) para descubrir patrones, tendencias y relaciones en los datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from scipy import stats

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carga de Datos Limpios

Cargamos el dataset previamente limpiado desde la carpeta processed. Convertimos inmediatamente ORDERDATE a formato datetime para permitir operaciones de series temporales. Verificamos el rango de fechas para entender el periodo de análisis.

In [ ]:
df = pd.read_csv('../data/processed/sales_data_clean.csv')
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['ORDERDATE'].min()} to {df['ORDERDATE'].max()}")

## 2. Análisis Univariado

El análisis univariado examina cada variable de forma independiente para entender su distribución, centralidad y dispersión.

### 2.1 Variables Numéricas

Analizamos la distribución de variables cuantitativas usando estadísticas descriptivas y histogramas. Las líneas de media y mediana nos permiten identificar si las distribuciones son simétricas o tienen sesgo. Esto es fundamental para decidir qué técnicas estadísticas aplicar posteriormente.

In [ ]:
numeric_cols = ['QUANTITYORDERED', 'PRICEEACH', 'SALES', 'MSRP']
df[numeric_cols].describe()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    axes[idx].axvline(df[col].median(), color='green', linestyle='--', label='Median')
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../results/distributions_numerical.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.2 Variables Categóricas

Examinamos la frecuencia y distribución de variables cualitativas. Identificamos las categorías más comunes en cada variable, lo cual es esencial para entender la composición del dataset y detectar posibles desequilibrios en los datos que podrían afectar análisis posteriores.

In [ ]:
categorical_cols = ['STATUS', 'PRODUCTLINE', 'DEALSIZE', 'COUNTRY', 'TERRITORY']

for col in categorical_cols:
    print(f"\n{col} - Unique values: {df[col].nunique()}")
    print(df[col].value_counts().head(10))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    top_values = df[col].value_counts().head(10)
    axes[idx].bar(range(len(top_values)), top_values.values)
    axes[idx].set_xticks(range(len(top_values)))
    axes[idx].set_xticklabels(top_values.index, rotation=45, ha='right')
    axes[idx].set_title(f'Top {col}')
    axes[idx].set_ylabel('Count')

axes[-1].axis('off')
plt.tight_layout()
plt.savefig('../results/distributions_categorical.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Análisis Bivariado

El análisis bivariado explora relaciones entre pares de variables para identificar asociaciones y patrones.

### 3.1 Ventas por Línea de Producto

Agrupamos las ventas por PRODUCTLINE para identificar qué categorías generan más ingresos. Calculamos tres métricas: suma total (volumen de negocio), promedio (ticket medio) y conteo (volumen de órdenes). Estas métricas combinadas revelan tanto la escala como la eficiencia de cada línea de producto.

In [ ]:
sales_by_product = df.groupby('PRODUCTLINE')['SALES'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)
sales_by_product.columns = ['Total_Sales', 'Avg_Sales', 'Orders']
print(sales_by_product)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].bar(sales_by_product.index, sales_by_product['Total_Sales'])
axes[0].set_title('Total Sales by Product Line')
axes[0].set_xlabel('Product Line')
axes[0].set_ylabel('Total Sales ($)')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(sales_by_product.index, sales_by_product['Avg_Sales'])
axes[1].set_title('Average Sales by Product Line')
axes[1].set_xlabel('Product Line')
axes[1].set_ylabel('Average Sales ($)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../results/sales_by_productline.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.2 Ventas por País/Territorio

Analizamos el desempeño geográfico del negocio. El análisis por país identifica mercados específicos más rentables, mientras que el análisis por territorio (EMEA, NA, APAC, Japan) muestra el desempeño a nivel regional. Esto es crítico para estrategias de expansión y asignación de recursos.

In [ ]:
sales_by_country = df.groupby('COUNTRY')['SALES'].sum().sort_values(ascending=False).head(15)

plt.figure(figsize=(12, 6))
plt.barh(sales_by_country.index, sales_by_country.values)
plt.xlabel('Total Sales ($)')
plt.title('Top 15 Countries by Sales')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../results/sales_by_country.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sales_by_territory = df.groupby('TERRITORY')['SALES'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
plt.pie(sales_by_territory.values, labels=sales_by_territory.index, autopct='%1.1f%%', startangle=90)
plt.title('Sales Distribution by Territory')
plt.savefig('../results/sales_by_territory.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.3 Análisis de Series Temporales

Estudiamos la evolución de las ventas a lo largo del tiempo. Creamos la variable YEAR_MONTH agrupando por periodo mensual para identificar tendencias y estacionalidad. El análisis temporal revela patrones de crecimiento, decrecimiento y ciclos que son fundamentales para pronósticos y planificación estratégica.

In [ ]:
df['YEAR_MONTH'] = df['ORDERDATE'].dt.to_period('M')
monthly_sales = df.groupby('YEAR_MONTH')['SALES'].sum()

plt.figure(figsize=(14, 6))
monthly_sales.plot(kind='line', marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/monthly_sales_trend.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sales_by_year = df.groupby('YEAR_ID')['SALES'].sum()
sales_by_quarter = df.groupby(['YEAR_ID', 'QTR_ID'])['SALES'].sum().reset_index()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].bar(sales_by_year.index, sales_by_year.values)
axes[0].set_title('Annual Sales')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Total Sales ($)')

for year in sales_by_quarter['YEAR_ID'].unique():
    year_data = sales_by_quarter[sales_by_quarter['YEAR_ID'] == year]
    axes[1].plot(year_data['QTR_ID'], year_data['SALES'], marker='o', label=f'Year {year}')

axes[1].set_title('Quarterly Sales by Year')
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Total Sales ($)')
axes[1].legend()
axes[1].set_xticks([1, 2, 3, 4])

plt.tight_layout()
plt.savefig('../results/annual_quarterly_sales.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.4 Análisis por Tamaño de Negociación

Examinamos cómo se comportan las ventas según el tamaño del deal (Small, Medium, Large). Calculamos estadísticas por categoría y usamos boxplots para comparar distribuciones. Esto ayuda a entender si los grandes deals tienen características distintivas y a optimizar estrategias de pricing.

In [ ]:
dealsize_stats = df.groupby('DEALSIZE').agg({
    'SALES': ['sum', 'mean', 'count'],
    'QUANTITYORDERED': 'mean'
}).round(2)

print(dealsize_stats)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

df.boxplot(column='SALES', by='DEALSIZE', ax=axes[0])
axes[0].set_title('Sales Distribution by Deal Size')
axes[0].set_xlabel('Deal Size')
axes[0].set_ylabel('Sales ($)')
plt.suptitle('')

dealsize_counts = df['DEALSIZE'].value_counts()
axes[1].pie(dealsize_counts.values, labels=dealsize_counts.index, autopct='%1.1f%%')
axes[1].set_title('Distribution of Deal Sizes')

plt.tight_layout()
plt.savefig('../results/dealsize_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Análisis Multivariado

El análisis multivariado examina relaciones entre tres o más variables simultáneamente.

### 4.1 Análisis de Correlaciones

Calculamos la matriz de correlación de Pearson entre variables numéricas. El heatmap visualiza la fuerza y dirección de las relaciones lineales. Correlaciones fuertes (cercanas a 1 o -1) indican que variables se mueven juntas, información valiosa para modelado predictivo y detección de multicolinealidad.

In [ ]:
numeric_features = ['QUANTITYORDERED', 'PRICEEACH', 'SALES', 'MSRP']
correlation_matrix = df[numeric_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('../results/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

### 4.2 Análisis Cruzado: Línea de Producto vs Territorio

Creamos una tabla dinámica (pivot table) que cruza dos variables categóricas para ver ventas totales en cada combinación. El heatmap resultante identifica qué productos funcionan mejor en qué regiones, permitiendo estrategias de marketing geográficamente segmentadas.

In [ ]:
pivot_table = df.pivot_table(values='SALES', index='PRODUCTLINE', 
                              columns='TERRITORY', aggfunc='sum', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlOrRd', linewidths=0.5)
plt.title('Sales by Product Line and Territory')
plt.tight_layout()
plt.savefig('../results/productline_territory_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

### 4.3 Análisis de Principales Clientes

Identificamos los 20 clientes más valiosos agregando ventas por CUSTOMERNAME. El scatter plot relaciona número de órdenes con ventas totales, revelando diferentes perfiles: clientes de alto volumen/alta frecuencia vs clientes de alto valor/baja frecuencia. Esta segmentación es clave para estrategias CRM.

In [ ]:
top_customers = df.groupby('CUSTOMERNAME').agg({
    'SALES': 'sum',
    'ORDERNUMBER': 'nunique'
}).sort_values('SALES', ascending=False).head(20)

top_customers.columns = ['Total_Sales', 'Number_of_Orders']
print("Top 20 Customers:")
print(top_customers)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

axes[0].barh(range(len(top_customers)), top_customers['Total_Sales'])
axes[0].set_yticks(range(len(top_customers)))
axes[0].set_yticklabels(top_customers.index)
axes[0].set_xlabel('Total Sales ($)')
axes[0].set_title('Top 20 Customers by Sales')
axes[0].invert_yaxis()

axes[1].scatter(top_customers['Number_of_Orders'], top_customers['Total_Sales'], alpha=0.6, s=100)
axes[1].set_xlabel('Number of Orders')
axes[1].set_ylabel('Total Sales ($)')
axes[1].set_title('Customer Orders vs Sales')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/top_customers_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Análisis de Estado de Órdenes

Analizamos la distribución de STATUS para entender la tasa de cumplimiento de órdenes. Calculamos métricas agregadas por estado (Shipped, Cancelled, Disputed, etc.) para identificar problemas operativos. Órdenes no completadas representan ingresos perdidos y oportunidades de mejora de procesos.

In [ ]:
status_analysis = df.groupby('STATUS').agg({
    'ORDERNUMBER': 'count',
    'SALES': ['sum', 'mean']
}).round(2)

print("Order Status Summary:")
print(status_analysis)

In [ ]:
status_counts = df['STATUS'].value_counts()

plt.figure(figsize=(10, 6))
plt.bar(status_counts.index, status_counts.values)
plt.title('Order Status Distribution')
plt.xlabel('Status')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../results/order_status_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Exportación de Estadísticas Resumen

Compilamos métricas clave del negocio en un diccionario: ventas totales, número de órdenes únicas, clientes únicos, valor promedio de orden, productos únicos, países y rango de fechas. Estas estadísticas se exportan a CSV para uso en reportes ejecutivos y dashboards.

In [ ]:
summary_stats = {
    'total_sales': df['SALES'].sum(),
    'total_orders': df['ORDERNUMBER'].nunique(),
    'total_customers': df['CUSTOMERNAME'].nunique(),
    'avg_order_value': df.groupby('ORDERNUMBER')['SALES'].sum().mean(),
    'total_products': df['PRODUCTCODE'].nunique(),
    'countries': df['COUNTRY'].nunique(),
    'date_range': f"{df['ORDERDATE'].min()} to {df['ORDERDATE'].max()}"
}

print("\n" + "="*50)
print("EDA SUMMARY STATISTICS")
print("="*50)
for key, value in summary_stats.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

pd.Series(summary_stats).to_csv('../results/summary_statistics.csv')
print("\nSummary statistics saved to results/summary_statistics.csv")